In [1]:
!pip install -q gpt_2_simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import json
import numpy as np
import random
from os import listdir
from os.path import isfile, join, splitext
import tensorflow as tf
import nltk
import nltk.sentiment.vader as vader
import shutil

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
gpt2.download_gpt2()

Fetching checkpoint: 1.00kit [00:00, 297kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 48.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 712kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:14, 35.0Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 2.58Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 46.5Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 35.8Mit/s]                                                       


In [0]:
gpt2.mount_gdrive()
gpt2.copy_checkpoint_from_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
sid = vader.SentimentIntensityAnalyzer()

In [0]:
def sample_random(sess, prefix, temp=0.7):
  return gpt2.generate(sess, length=len(prefix), temperature=temp, prefix=prefix)

def sample_sentiment(sess, prefix, sentiment_vec, max_iter=100, thresh=0):
  """ 
  Sentiment based rejection sampling
  returns the best sample
  Sample either hundred times or when we do better than threshold distance
  note thresh is between [0,2]. It's an early cutoff
  """
  best_score = 2
  best_result = None
  for i in range(max_iter):
    sample = sample_random(sess, prefix, random.random())
    if not sample:
      continue
    ss = sid.polarity_scores(sample)
    curr_sent = np.array([ss["compound"], ss["neg"], ss["neu"], ss["pos"]])
    dist = np.linalg.norm(sentiment_vec-curr_sent)
    if (dist < thresh):
      return sample
    if dist < best_score:
      best_result = sample
      best_score = dist
  return best_result

In [0]:
def test_single(sid, train_file, test_instance, save_dest):
  """
  Takes the single testing JSON file as a dict.
  Runs sampling on all 3, saves results to dest dir, each training example to
  its own file
  """
  tf.reset_default_graph()
  sess = gpt2.start_tf_sess()
#   shutil.copyfile('drive/My Drive/checkpoint/run1/checkpoint', 'checkpoint/run1/checkpoint')
  
  gpt2.finetune(sess,
                dataset=train_file,
                steps=10,
                print_every=5,
                sample_every=1000,
                sample_length=1,
                restore_from='fresh',
                save_every=30,
                ) # don't want to save
  print('finetune done')
  results = []
  for i in range(1,4):
    inp = test_instance["test{}_input".format(i)]
    tru = test_instance["test{}_true".format(i)]
    id_ = test_instance["test{}_id".format(i)]
    # Generate random sample
    samp_rand = sample_random(sess, inp)
    # Generate sentiment sample
    ss = sid.polarity_scores(tru)
    sentiment_vec = np.array([ss["compound"], ss["neg"], ss["neu"], ss["pos"]])
    samp_sent = sample_sentiment(sess, inp, sentiment_vec)
    sample = {"id": id_,
              "prefix": inp,
              "truth": tru,
              "random": samp_rand,
              "sentiment": samp_sent
             }
    results.append(sample)
  with open(save_dest, "w") as f:
    json.dump(results, f)
  print('finished test single')

def run_tests(test_dir, train_dir):
  # Load model from checkpoint
  for f in listdir(train_dir):
    fname = splitext(f)[0]
    train = join(train_dir, f)
    test = join(test_dir, fname+".json")
    if isfile(train) and isfile(test):
      with open(test, "r") as test_file:
        test_instance = json.load(test_file)
        print(f'Testing {fname}')
        test_single(sid, train, test_instance, "/content/results/{}_sample.json".format(f))

In [13]:
run_tests('moviescriptmining/test_cases', 'moviescriptmining/training_cases')

Testing Martian,-The
Loading checkpoint models/117M/model.ckpt
INFO:tensorflow:Restoring parameters from models/117M/model.ckpt


100%|██████████| 1/1 [00:00<00:00,  5.64it/s]

Loading dataset...
dataset has 17074 tokens
Training...


[5 | 18.94] loss=2.75 avg=2.75
[10 | 31.53] loss=2.48 avg=2.61
Saving checkpoint/run1/model-10
finetune done
I. HAB - DAY 
Mark cuts each potato into four quarters, making sure each quarter has at least two eyes. 
He begins planting each potato quarter in nice, orderly rows. As he works, we slowly WIDEN OUT to reveal -- 
The ENTIRE HAB is now covered in SOIL. Not just the floor -- Mark has cleared every available surface -- bunks, countertops, table -- and covered it with his dirt. 
INT. HAB - DAY 
Mark packs soil on top of one of the crew member's bunks. As he moves the personal items aside, he finds a DATA-STICK. He holds it up and looks at it: hmmmm. CUT TO: 
Mark has plugged the data-stick into the computer and is now viewing its contents: old episodes of seventies television. 
Mark just sits there. Watching HAPPY DAYS. 
TITLE: SOL 36 
MARK (PRELAP) The problem is water... 
INT./EXT - ROVER - DAY 
Mark trudges out to the Mars Descent Vehicle (MDV) -- the lander that brought the six

KeyboardInterrupt: ignored